# Response Generation and Comparison (Flan-T5 vs Zephyr)
This notebook takes customer review texts, finds similar examples using FAISS, and generates a short, friendly reply using either Flan-T5-small or Zephyr-7b-beta.

It then compares the responses using human-written references and evaluates them with BLEU, ROUGE-L, and Perplexity.

We use GPT-2 to estimate the **perplexity** of generated responses, which helps assess their fluency and coherence.


### Load LoRA Classification Model + Predictions CSV
We load the fine-tuned classification model (LoRA) and the CSV with predicted labels, extracted from a shared ZIP package.


In [ ]:
# Upgrade to the latest version of bitsandbytes for 4-bit quantization support
!pip install -q --upgrade bitsandbytes

In [ ]:
import subprocess

# List of all required packages
all_packages = [
    "bitsandbytes",                   # For 4-bit quantization (Zephyr)
    "faiss-cpu",                      # For fast similarity search
    "sentence-transformers",          # For embeddings
    "evaluate"                        # For BLEU, ROUGE, etc.
]

# Unified silent pip install
command = ["pip", "install", "-q"] + all_packages
result = subprocess.run(command, capture_output=True, text=True)

# Optional: final check message
if result.returncode == 0:
    print(" All required packages installed successfully.")
else:
    print(" Installation failed:\n", result.stderr)



In [ ]:
# ===========================
# Library Imports – Generation Pipeline
# ===========================

# Standard libraries
import os                     # File/directory operations
import zipfile                # For unzipping the model/data archive
import pandas as pd           # Data manipulation (DataFrames)
import numpy as np            # Numerical operations

# PyTorch
import torch                  # Tensor operations (used by Transformers)

# ===========================
# Hugging Face Transformers
# ===========================

# Tokenizer and model for classification (LoRA)
from peft import PeftModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Flan-T5 model for sequence-to-sequence generation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Zephyr model for causal generation (instruction-tuned model)
from transformers import AutoModelForCausalLM, AutoTokenizer

# GPT-2 model for evaluating perplexity
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# ===========================
# Evaluation Libraries
# ===========================

from evaluate import load     # For BLEU, ROUGE-L, etc.

# ===========================
# FAISS + Embedding Models
# ===========================

import faiss                  # Fast similarity search on embeddings
from sentence_transformers import SentenceTransformer  # To encode review texts


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Unzip the saved package
with zipfile.ZipFile("bert_sentiment_package.zip", 'r') as zip_ref:
    zip_ref.extractall("app")
    print(" Contenu de l'archive ZIP :")
    print(zip_ref.namelist())

In [ ]:
# Function to load LoRA fine-tuned model and tokenizer
def load_classification_model():
    base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    model = PeftModel.from_pretrained(base_model, "app/bert_sentiment_lora")
    tokenizer = AutoTokenizer.from_pretrained("app/bert_sentiment_lora")
    model.eval()
    return tokenizer, model


In [ ]:
# # Load model and CSV
# cls_tokenizer, cls_model = load_classification_model()
test_df = pd.read_csv("app/test_with_predictions.csv")
print(" LoRA model and CSV loaded.")
test_df.head(2)

### Load Flan-T5 model for generation

 We start by loading a lightweight T5 model fine-tuned by Google for general instruction-following tasks.

In [ ]:
# Load Flan-T5-small tokenizer and model
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### Setup FAISS Index and SentenceTransformer

We encode all the clean_texts  and build a FAISS index for fast nearest neighbor search.


In [ ]:
# Load lightweight encoder model
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
df = pd.read_csv("app/test_with_predictions.csv")

# Extract texts to index (use df["clean_text"] or "clean_combined")
texts = df["text"].tolist()

# Encode texts into embeddings (N x 384)
embeddings = encoder.encode(texts, show_progress_bar=True, convert_to_numpy=True)

# Create FAISS cosine similarity index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)
index.add(embeddings)

print(f"FAISS index created with {index.ntotal} vectors.")


###  Function to Find Similar Reviews

Given a customer comment, we search the FAISS index to retrieve the top-k most similar reviews.


In [ ]:
def search_similar(user_comment, top_k=3):
    # Encode the user comment
    query_embedding = encoder.encode([user_comment], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)

    # Search for top-k most similar reviews
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the matching texts
    similar_texts = [texts[i] for i in indices[0]]

    print(" User comment:")
    print(user_comment)
    print("\n Similar reviews found:\n")
    for i, text in enumerate(similar_texts, 1):
        print(f"{i}. {text}\n")

    return similar_texts


### 💡 Sentiment-Aware Generation

### In this version of the pipeline, we incorporate the **predicted sentiment** of each user comment to help the language model generate more appropriate replies.

### This bridges our two tasks:
 - **Sentiment classification (Notebook 1)** trained using a BERT-based model
- **Response generation (Notebook 2)** using Flan-T5 and Zephyr

### For each comment:
 - We retrieve the predicted sentiment (`positive` or `negative`)
 - We search for similar reviews using FAISS
 - We build a context-enriched prompt with the sentiment explicitly added
 - We generate a tailored reply from each model


###  Prompt Construction Functions

These functions format prompts differently for Flan-T5-small and Zephyr-7b-beta.

They include the user comment and retrieved similar reviews as context.


In [ ]:
def build_prompt_with_sentiment(user_comment, sentiment, similar_texts):
    context = "\n".join([f"{i+1}. {text}" for i, text in enumerate(similar_texts)])
    prompt = (
        f"You are a customer support assistant at SanDisk.\n"
        f"The user's sentiment is **{sentiment.upper()}**.\n\n"
        f"Based on their comment and similar reviews, write a short, friendly, and helpful reply.\n"
        f"Tone should match the sentiment: empathetic if negative, encouraging if positive.\n"
        f"Keep the response under 3 sentences.\n\n"
        f"User comment:\n{user_comment}\n\n"
        f"Similar reviews:\n{context}\n\n"
        f"Reply:"
    )
    return prompt
# Build prompt for Zephyr with sentiment
def build_prompt_zephyr_with_sentiment(user_comment, sentiment, similar_reviews):
    context = "\n".join([f"{i+1}. {rev}" for i, rev in enumerate(similar_reviews)])
    prompt = (
        f"You are an Amazon customer service assistant.\n"
        f"The sentiment of the review is **{sentiment.upper()}**.\n"
        f"Write a short and casual reply to the following customer review (max 2 sentences).\n"
        f"Be empathetic if the sentiment is negative, and upbeat if positive.\n\n"
        f"Customer review:\n{user_comment}\n\n"
        f"Similar reviews:\n{context}\n\n"
        f"Reply:"
    )
    return prompt

Flan-T5 Response Generation

In [ ]:
# Flan-T5 generation with sentiment-aware prompt
def generate_response_flan(prompt, max_length=150):
    inputs = flan_tokenizer(prompt, return_tensors="pt", truncation=True)
    output = flan_model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=1
    )
    return flan_tokenizer.decode(output[0], skip_special_tokens=True)

Load Zephyr Model and Pipeline


In [ ]:
model_id = "HuggingFaceH4/zephyr-7b-beta"

zephyr_tokenizer = AutoTokenizer.from_pretrained(model_id)
zephyr_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)



In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=zephyr_model,
    tokenizer=zephyr_tokenizer,
    device_map="auto",
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)


Zephyr Response Generator (with cleaner reply extraction)

In [ ]:
def generate_response_zephyr(prompt, max_length=100):
    raw_output = pipe(prompt, max_new_tokens=max_length)[0]["generated_text"]
    reply = raw_output.split("Reply:")[-1].strip()

    # Split at "Or:" or duplicate variants if needed
    reply = reply.split("\nOr:")[0].strip()
    return reply


Comparative Reply Generation (with Sentiment)


In [ ]:
comparative_data = []

for _, row in df.sample(3, random_state=42).iterrows():
    user_comment = row["text"]
    sentiment = row["predicted_sentiment"]

    similar_reviews = search_similar(user_comment)

    flan_prompt = build_prompt_with_sentiment(user_comment, sentiment, similar_reviews)
    flan_reply = generate_response_flan(flan_prompt)

    zephyr_prompt = build_prompt_zephyr_with_sentiment(user_comment, sentiment, similar_reviews)
    zephyr_reply = generate_response_zephyr(zephyr_prompt)

    comparative_data.append({
        "User Comment": user_comment,
        "Sentiment": sentiment,
        "Flan-T5 Reply": flan_reply,
        "Zephyr Reply": zephyr_reply
    })

In [ ]:
 comparison_df = pd.DataFrame(comparative_data)
 pd.set_option('display.max_colwidth', None)
# comparison_df.head()

### Perplexity-only Evaluation (Flan and Zephyr Replies)
This step computes the fluency of generated replies from both Flan-T5 and Zephyr using the GPT-2 model.

The lower the perplexity, the more fluent and natural the response is.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load GPT-2 model and tokenizer for perplexity scoring
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
gpt2_model.eval()

def calculate_perplexity(text):
    inputs = gpt2_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = gpt2_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    return torch.exp(loss).item()


In [ ]:
# Compute perplexity for both Flan and Zephyr replies
flan_perplexities = [calculate_perplexity(reply) for reply in comparison_df["Flan-T5 Reply"]]
zephyr_perplexities = [calculate_perplexity(reply) for reply in comparison_df["Zephyr Reply"]]

# Add results to the DataFrame
comparison_df["Flan Perplexity"] = flan_perplexities
comparison_df["Zephyr Perplexity"] = zephyr_perplexities

# Preview final comparison table
comparison_df[["User Comment", "Sentiment", "Flan-T5 Reply", "Flan Perplexity", "Zephyr Reply", "Zephyr Perplexity"]]


### Analysis of Perplexity Results

We evaluated the fluency of generated replies using **GPT-2 perplexity scores**:

- **Lower perplexity = more fluent and natural text.**

#### Observations:
- Zephyr consistently achieves lower perplexity scores (≈13–20), indicating smoother and more coherent replies.
- Flan-T5 shows mixed results: while its output is shorter, it sometimes lacks context or generates incoherent text.
- Example 2 from Flan ("Great microsd card.") has an extremely high perplexity (268), likely due to the short, out-of-context sentence.

**Zephyr performs better in terms of fluency**, especially when combining context and sentiment.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "HuggingFaceH4/zephyr-7b-beta"

zephyr_tokenizer = AutoTokenizer.from_pretrained(model_id)
zephyr_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cpu")

zephyr_tokenizer.save_pretrained("zephyr_generator_fp32")
zephyr_model.save_pretrained("zephyr_generator_fp32")


In [ ]:
!zip -r zephyr_generator_fp32.zip zephyr_generator_fp32
from google.colab import files
files.download("zephyr_generator_fp32.zip")